# 6.1 Reading and Writing Data in Text Format

1. [Basic Reading](#basic)
1. [Reading Text Files in Pieces](#reading)
1. [Writing Data to Text Format](#writing)
1. [Working with Other Delimited Formats](#other)
1. [JSON Data](#json)
1. [XML and HTML: Web Scraping](#web)

<a name="basic"></a>
# Basic Reading

There are lots of ways to read data into Python with pandas.  

`pands.read_csv` is the most common and will be used most in this tutorial.

<img src="./myImages/table6.1_loadingFunctions.png" width = 600>

All of the above functions are intended to convert the data contained in them into a usable DataFrame.  

The default behavior can be modified based on various arguments.  

Some common considerations:

1. **Indexing**
    - specify which columns to include in the DataFrame
    - specify if column names come from file, arguments given, or no names
1. **Type inference and data conversion**
    - If you have a custom list of missing values and/or specific user-defined value conversions
1. **Date and time parsing**
    - Can combine date/time data from multiple columns into 1
1. **Iterating**
    -Iterating over chunks of very large files
1. **Unclean data issues**
    - skipping rows/footer
    - skipping comments
    - handle large numbers with commas
    - etc.

## Read in File with header

In [107]:
import pandas as pd
import numpy as np

# Example CSV file
!cat ../../examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [108]:

df = pd.read_csv("../../examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Read in File with no header

In [109]:
!cat ../../examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [110]:
# Read in with default names
df2_default = pd.read_csv("../../examples/ex2.csv", header=None)
df2_default

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [111]:
# Read in and supply names
df2_user = pd.read_csv("../../examples/ex2.csv", names=["a", "b", "c", "d", "message"])
df2_user

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Specify Index Column

In [112]:
# With default column names
df2_index1 = pd.read_csv("../../examples/ex2.csv", header=None, index_col=4)
df2_index1

,0,1,2,3
4,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [113]:
# With supplied names
colNames = ["a", "b", "c", "d", "message"]
df2_index2 = pd.read_csv("../../examples/ex2.csv", names=colNames, index_col="message")
df2_index2

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Hierarchical index will be explained more in Ch8

In [114]:
!cat ../../examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [115]:
parsed = pd.read_csv("../../examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

## Specify Delimiter

Can supply `\t` and other common ones. In this example, variable amounts of whitespace require `\s+`.  

Also notice how the row indices were assumed here b/c row 1 has 3 columns while all other rows have 4 columns.

In [116]:
!cat ../../examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [117]:
result = pd.read_csv("../../examples/ex3.txt", sep="\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


## Skip specific rows

In [118]:
!cat ../../examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [119]:
pd.read_csv("../../examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Missing Values

pandas recognizes a set of commonly occuring missing value "sentinels", notably NA, NULL, and blank

pands will convert them to NaN.  

Can also add na values to search for with `na_values` argument

In [120]:
!cat ../../examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [121]:
result = pd.read_csv("../../examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [122]:
pd.read_csv("../../examples/ex5.csv", na_values=["NULL"])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


The `keep_default_na` argument is a logical indicating if the default pandas behavior should be followed or not.  

If this is set to `False`, then the values in the file will not be parsed to `NaN` and will be left as is:

In [123]:
result2 = pd.read_csv("../../examples/ex5.csv", keep_default_na=False)
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


Can see that these cells are no longer considered NAs:

In [124]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


`keep_default_na=False` can be useful if you want to ignore some default NA conversions and only do some:

In [125]:
result3 = pd.read_csv("../../examples/ex5.csv", keep_default_na=False, na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [126]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


Can also specify different ones per column if you have a complicated table

In [127]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("../../examples/ex5.csv", na_values=sentinels,
            keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


<a name="reading"></a>
# Reading Text Files in Pieces

Two main reasons you might want to read files in pieces:
1. Break up large files to not overwhelm memory
2. Just get a sample to make sure you're reading it in correctly

In [128]:
# First change max row display option (this doesn't effect output in jupyter b/c already limited to 10 lines)
pd.options.display.max_rows = 10

# Read in large example file
result = pd.read_csv("../../examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [129]:
# Read in 5 rows only
pd.read_csv("../../examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [130]:
# Read file in chunks of 1000 lines
chunker = pd.read_csv("../../examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

The `chunker` object is a `TextFileReader` object and can now be iterated over with a for loop.  

Below we count the values of each `key` in each chunk

In [131]:
# Have to re-read chunker b/c we "used it up" with the type call
chunker = pd.read_csv("../../examples/ex6.csv", chunksize=1000)

# Empty Series to hold results
keyTotals = pd.Series([], dtype="int64")

# Count up each key value in each chunk
for piece in chunker:
    keyTotals = keyTotals.add(piece["key"].value_counts(), fill_value=0)
    
# Sort values and view
keyTotals = keyTotals.sort_values(ascending=False)
keyTotals

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

<a name="writing"></a>
# Writing Data to Text Format

The `to_csv` method is the standard method to use here.  
1. The default uses commas as the delimiter, but can be changed with `sep` argument.  
1. By default the column names and row indexes are written, but `header=False` and `index=False` will not, respectively.  
1. NA values are empty strings by default, but can specify what you want them to be with `na_rep` argument.
1. `columns` allows you to just write out a subset of columns.  

Another convenience displayed in the examples below is the `stdout` method of the `sys` module. This can be used to print the output to stdout instead of writing to a file.

In [132]:
# Read in example data
data = pd.read_csv("../../examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [133]:
# Write it out with defaults
data.to_csv("../../examples/out.csv")

# View it
!cat ../../examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [134]:
import sys

# Write with different delimiter
data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [135]:
# Assign NA value output
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [136]:
# Exclude both row and column labels
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [137]:
# Write only some columns
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


<a name="other"></a>
# Working with Other Delimited Formats

Sometimes you have to do some manual processing to a file before you're able to read it in with `pandas.read_csv`.  

The `csv` module allows you to read in in a similar manner as the base `read` option. Instead of reading in each line, however, this module is built to consider a single-character delimiter as well.  But it works on an opened file the same way.  

In the below example, we don't have a header or row index which will trip up `read_csv`

In [138]:
!cat ../../examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [139]:
import csv

# Open file connection
f = open("../../examples/ex7.csv")

# Load into an iterable csv.reader object
reader = csv.reader(f)
type(reader)

_csv.reader

In [140]:
# Loop over this iterator to remove the quotes
for line in reader:
    print(line)
    
# Close file
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [141]:
# Now fix it
# Remember the `with` functionality makes it so we 
# don't have to remember to close the file later
with open("../../examples/ex7.csv") as f:
    lines = list(csv.reader(f))                             # Load the file and coerce to a list

In [142]:
# Now we have a list, one element per line
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [143]:
# Use tuple evaluation to split into header and values
header, values = lines[0], lines[1:]

In [144]:
# Use dictionary comprehension along with zip to create a dictionary
# "h:v" for "h,v"
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

`csv.Dialect` is useful for combining specifc csv attributes

<img src="./myImages/table6.2_readCSVArgs.png" width=600>

In [145]:
class my_dialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

f = open("../../examples/ex7.csv")
reader = csv.reader(f, dialect=my_dialect)
f.close()

Writing with `csv.writer` is tedious

In [146]:
with open("../../examples/mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

<a name="json"></a>
# JSON Data

JSON (JavaScript Object Notation) is one of the standard formats for sending data by HTTP request between web browsers and other applications.  

It's more free-form than CSV.  

It's almost valid Python code, barring a few differences
    1. null value is `null`
    1. No trailing commas in lists

JSON types:
    1. objects (python dictionaries)
        - all keys must be strings
    1. arrays (python lists)
    1. strings, numbers, Booleans (all same as python)
    1. nulls

Various libraries for parsing JSON, `json` is in the standard library.

In [147]:
import json

### Example JSON object
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

### Convert a JSON string to Python
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [148]:
# Convert a python object back to JSON
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

Will have to decide for yourself how to convert to a DataFrame or whatever other structure you want to use for data analysis. Depends on the type of JSON object you're dealing with.  

1. `pandas.DataFrame` can handle a list of dictionaries
1. `pandas.read_json` will automatically convert **some** JSON datasets
    - default assumes each object in JSON array is a row in the table
1. DataFrames and Series have the `to_json` method that will convert to JSON

In [149]:
# Using DataFrame
allSiblings = pd.DataFrame(result["siblings"])
cutSiblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
print(allSiblings)
print("\n")
print(cutSiblings)

    name  age            hobbies
0  Scott   34  [guitars, soccer]
1  Katie   42      [diving, art]


    name  age
0  Scott   34
1  Katie   42


In [150]:
# Using read_json
!cat ../../examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [151]:
data = pd.read_json("../../examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [152]:
# Export from pandas to JSON
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [153]:
# Export by row
data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

<a name="web"></a>
# XML and HTML: Web Scraping

There are many libraries for reading and writing in HTML and XML:

1. `lxml`
1. `Beautiful Soup`
1. `html5lib`

`lxml` is generally the fastest, the other two are a bit better at malformed data.

pandas itself has a built-in function `pandas.read_html` that will read in html files. It depends upon the above libraries, so make sure they're installed.

Default behavior of `pandas.read_html` is to search for all tabular data within `<table>` tags and parse them into individual DataFrame objects that are combined into a list.

In [154]:
tables = pd.read_html("../../examples/fdic_failed_bank_list.html")
print(type(tables))
print(len(tables))

<class 'list'>
1


In [155]:
# Only have 1 table, so assign it
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [156]:
# Calculate number of bank failures by year
close_timestamps = pd.to_datetime(failures["Closing Date"])
print(close_timestamps.head())
close_timestamps.dt.year.value_counts()

0   2016-09-23
1   2016-08-19
2   2016-05-06
3   2016-04-29
4   2016-03-11
Name: Closing Date, dtype: datetime64[ns]


Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

## Parsing XML is lxml.objectify

Main function to parse xml files.

In [157]:
from lxml import objectify
path = "../../datasets/mta_perf/Performance_MNR.xml"

In [158]:
# Parse XML file
with open(path) as f:
    parsed = objectify.parse(f)
    
print(type(parsed))
print(str(parsed))

<class 'lxml.etree._ElementTree'>


In [159]:
# Get the root node
root = parsed.getroot()
root

<Element PERFORMANCE at 0x155caf280>

Use `root.INDICATOR` to get all of the `<INDICATOR>` XML elements. A generator is created.

In [160]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [161]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


Can do it even more easily with `pandas.read_xml`. This function has a bunch of different options to help with more complicated XML files.

In [162]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80
